In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import sys
import random

from tqdm import tqdm

import string

import os

import json

In [3]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 5.6 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 67.3 MB/s 
     |████████████████████████████████| 3.3 MB 56.6 MB/s 
     |████████████████████████████████| 596 kB 52.2 MB/s 


In [4]:
from transformers import AutoTokenizer, BertConfig, TFBertModel

In [5]:
import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy as sce
from tensorflow.keras.callbacks import Callback

In [6]:
train_ins = '/content/drive/MyDrive/data/fine_data/train_instance_192.csv'
val_san_ins = '/content/drive/MyDrive/data/fine_data/san_val_instance.csv'

val_id = '/content/drive/MyDrive/data/fine_data/val_id.csv'
train_id = '/content/drive/MyDrive/data/fine_data/train_id.csv'

gt = '/content/drive/MyDrive/data/fine_data/validation_short.csv'

## **Functions to tokenize data**

In [7]:
model_name = 'bert-base-uncased'

tokenizer_san = AutoTokenizer.from_pretrained(model_name)

tags = ['<Dd>', '<Dl>', '<Dt>', '<H1>', '<H2>', '<H3>', '<Li>', '<Ol>', '<P>', '<Table>', '<Td>', '<Th>', '<Tr>', '<Ul>',
        '</Dd>', '</Dl>', '</Dt>', '</H1>', '</H2>', '</H3>', '</Li>', '</Ol>', '</P>', '</Table>', '</Td>', '</Th>', '</Tr>', '</Ul>',
        '<Th_colspan=', '</Th_colspan=', '``', '\'\'', '--']

# for i in range(0, 51, 1):
#     tags.append(f'[part={i}]')

print(tags)

special_tokens_dict = {'additional_special_tokens': tags}

num_added_toks = tokenizer_san.add_special_tokens(special_tokens_dict)
print(num_added_toks)
print(len(tokenizer_san))

# encoder.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

['<Dd>', '<Dl>', '<Dt>', '<H1>', '<H2>', '<H3>', '<Li>', '<Ol>', '<P>', '<Table>', '<Td>', '<Th>', '<Tr>', '<Ul>', '</Dd>', '</Dl>', '</Dt>', '</H1>', '</H2>', '</H3>', '</Li>', '</Ol>', '</P>', '</Table>', '</Td>', '</Th>', '</Tr>', '</Ul>', '<Th_colspan=', '</Th_colspan=', '``', "''", '--']
33
30555


In [8]:
AnswerType = {
    'NO_ANSWER': 0,
    'YES': 1,
    'NO': 2,
    'SHORT' : 3,
    'LONG' : 4
}

def preprocess_data(data, tokenizer, debug=False): 
    progress = tqdm(data, total=len(data))
    x1 = []
    x2 = []
    x3 = []
    y = []
    for sam in progress:
        # part_id = sam['part_id']
        # part_tokens = f'[part={part_id}]'

        # context = part_tokens + " " + sam['context']
        tokenized_sam = tokenizer.encode_plus(sam['question'], sam['context'], 
                                              padding='max_length',
                                              truncation=True,
                                              max_length=512,
                                              add_special_tokens=True)
        
        x1.append(tf.cast(tokenized_sam['input_ids'], tf.int32))
        x2.append(tf.cast(tokenized_sam['token_type_ids'], tf.int32))
        x3.append(tf.cast(tokenized_sam['attention_mask'], tf.int32))

        y.append([sam['start'], sam['stop'], AnswerType[sam['target']]])

    x1 = tf.convert_to_tensor(x1)
    x2 = tf.convert_to_tensor(x2)
    x3 = tf.convert_to_tensor(x3)

    y = tf.convert_to_tensor(y)
    return x1, x2, x3, y


In [9]:
train_ins_df = pd.read_csv(train_ins)
tran_ins_list = train_ins_df.to_dict('records')
print(len(tran_ins_list))

432850


In [24]:
val_id_df = pd.read_csv(val_id)
val_id_list = val_id_df['example_id'].tolist()

gt_df = pd.read_csv(gt)
gt_list = gt_df.to_dict('records')

In [10]:
val_ins_df = pd.read_csv(val_san_ins)
val_ins_list = val_ins_df.to_dict('records')
print(len(val_ins_list))

2755


In [11]:
def get_strategy():
    try:
        tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print('Running on TPU ', tpu_cluster_resolver.cluster_spec().as_dict()['worker'])
        tf.config.experimental_connect_to_cluster(tpu_cluster_resolver)
        tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver)
        strategy = tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)
    except ValueError as e:
        print(e)
        print('No TPU detected')
        tpu = None
        strategy = tf.distribute.get_strategy()
    return strategy

In [12]:
def create_model_san(tokenizer, model_name, debug=False):
    config = BertConfig()
    if debug:
        print(config)
    # encoder = TFBertModel(config)
    encoder = TFBertModel.from_pretrained(model_name)
    encoder.resize_token_embeddings(len(tokenizer))

    NUM_TARGET = 5
    class MyQAModel(tf.keras.Model):
        def __init__(self, *inputs, **kwargs):
            super().__init__(*inputs, **kwargs)            
            self.bert = encoder

            # self.dropout_start = tf.keras.layers.Dropout(0.1)
            # self.dropout_stop = tf.keras.layers.Dropout(0.1)
            # self.dropout_target = tf.keras.layers.Dropout(0.1)

            self.start_logits = tf.keras.layers.Dense(1)
            self.stop_logits = tf.keras.layers.Dense(1)
            
            self.target = tf.keras.layers.Dense(NUM_TARGET)

        def call(self, inputs, **kwargs):
            bert_res=self.bert(inputs[0], 
                               token_type_ids=inputs[1], 
                               attention_mask=inputs[2]
                               )
            
            # dropout_res1 = self.dropout_start(bert_res[0])
            dropout_res1 = bert_res[0]

            start_logits = tf.squeeze(self.start_logits(dropout_res1), -1)

            # dropout_res2 = self.dropout_stop(bert_res[0])
            dropout_res2 = bert_res[0]

            stop_logits = tf.squeeze(self.stop_logits(dropout_res2), -1)

            # dropout_res3 = self.dropout_target(bert_res[1])
            dropout_res3 = bert_res[1]
            
            targets = self.target(dropout_res3)
            
            paddings = tf.constant([[0, 0,], [0, 512-NUM_TARGET]])
            targets = tf.pad(targets, paddings)
            
            res = tf.stack([start_logits, stop_logits, targets], axis=1)
            return res
        
    model = MyQAModel()
    return model 

In [13]:
def getSanFineInsRes(raw_res, ins_list, debug=False):
    list_ins_res = []
    progress = tqdm(ins_list, total=len(ins_list))
    for rowid, row in enumerate(progress):
        example_id = row['example_id']
        part_start = row['part_start']

        res_start = tf.nn.softmax(raw_res[rowid][0]).numpy()
        res_stop = tf.nn.softmax(raw_res[rowid][1]).numpy()
        res_target = tf.nn.softmax(raw_res[rowid][2]).numpy()
        
        start = np.argmax(res_start)
        stop = np.argmax(res_stop)
        target = np.argmax(res_target)

        start_score = res_start[start]
        stop_score = res_stop[stop]

        start_CLS = res_start[0]
        stop_CLS = res_stop[0]
        
        ins_res = {}
        ins_res['example_id'] = example_id
        ins_res['part_start'] = part_start

        ins_res['san_start'] = start 
        ins_res['san_stop'] = stop
        ins_res['san_target'] = target

        ins_res['start_score'] = start_score
        ins_res['stop_score'] = stop_score
        
        ins_res['start_CLS'] = start_CLS
        ins_res['stop_CLS'] = stop_CLS
        if debug:
            if rowid == 101:
                print(row)
                print(ins_res)
        list_ins_res.append(ins_res)
    list_ins_res_df = pd.DataFrame(list_ins_res)
    return list_ins_res_df

In [14]:
def mergeDocumentResSan(ins_df, val_id_df, threshold=0.0001, debug=False):
    list_doc_lan = []
    for idx, doc in val_id_df.iterrows():
        doc_id = doc['example_id']
        ins_of_doc = ins_df.loc[ins_df['example_id'] == doc_id]
        
        start_ins = ins_of_doc.loc[ins_of_doc['san_start'] != 0]
        stop_ins = ins_of_doc.loc[ins_of_doc['san_stop'] != 0]
        all_non_zero = pd.concat([start_ins,stop_ins]).drop_duplicates()
        
        best_start = -1
        best_stop = -1
        best_target = 0
        best_score = threshold
                    
        for idx_ins, ins in all_non_zero.iterrows():
            ins_start = int(ins['san_start'])
            ins_stop = int(ins['san_stop'])
            ins_target = int(ins['san_target'])
            
            part_start = ins['part_start']
            
            real_start = int(ins_start + part_start)
            real_stop = int(ins_stop + part_start)
            
            s_start = ins['start_score']
            s_stop = ins['stop_score']
            
            cls_start = ins['start_CLS']
            cls_stop = ins['stop_CLS']
            
            if real_stop > real_start:   
                if s_start - cls_start + s_stop - cls_stop > best_score:
                    best_score = s_start - cls_start + s_stop - cls_stop
                    best_start = real_start
                    best_stop = real_stop
                    best_target = ins_target

        doc_lan = {}
        doc_lan['example_id'] = doc_id
        doc_lan['san_start'] = best_start
        doc_lan['san_stop'] = best_stop
        doc_lan['san_target'] = best_target
        doc_lan['score'] = best_score
        
        if debug:
            if idx == 101:
                print(doc_lan)
        
        list_doc_lan.append(doc_lan)
    
    list_doc_lan_df = pd.DataFrame(list_doc_lan)
    return list_doc_lan_df

In [15]:
AnswerTypeRev = {
    0: 'NO_ANSWER',
    1: 'YES',
    2: 'NO',
    3: 'SHORT',
    4: 'LONG'
}

def getSanSubmission(doc_res_df, threshold=0.0001, debug=False):
    doc_res_df.example_id = doc_res_df.example_id.astype(str)
    lines = []
    for id, doc in doc_res_df.iterrows():
        example_id = doc['example_id']
        short_id = str(example_id) + '_short'

        line_short = {}
        line_short['example_id'] = short_id

        an_start = int(doc['san_start'])
        an_stop = int(doc['san_stop'])
        an_target = int(doc['san_target'])
        an_score = float(doc['score'])

        if an_start > 0 and an_stop > 0 and an_stop - an_start < 30:
            short_string = str(an_start) + ':' + str(an_stop)
        else:
            short_string = ''

        if an_target == 1 or an_target == 2:
            short_string = AnswerTypeRev[an_target]


        line_short['PredictionString'] = short_string
        lines.append(line_short)

    lines_df = pd.DataFrame(lines)
    sorted_df = lines_df.sort_values('example_id')
    return sorted_df

In [16]:
def getResult(gt_df, res_df, debug=False):
    gt_df.fillna('', inplace=True)
    if gt_df.shape[0] != res_df.shape[0]:
        print('ERROR: Different number of rows')
        return -1.0
    
    TP, TN, FP, FN = 0, 0, 0, 0

    # list_incorrect = []

    gt_id = gt_df['example_id'].astype(str).tolist()
    res_id = res_df['example_id'].astype(str).tolist()

    if gt_id != res_id:
        print("ERROR: Example_id lists are not the same")
        return -1.0
    
    gt_res = gt_df['PredictionString'].tolist()
    res_res = res_df['PredictionString'].tolist()
    id_list = gt_df['example_id'].tolist()
    for i in range(len(gt_res)):
        if gt_res[i] == res_res[i]:
            if gt_res[i] != "":
                TP += 1
            else:
                TN += 1
        else:
            if res_res[i] == '':
                FN += 1
            else:
                FP += 1
    recall = float(TP)/float(TP+FN + 0.000001)
    precision = float(TP)/float(TP+FP + 0.000001)
    print("TP: ", TP)
    print("TN: ", TN)
    print("FP: ", FP)
    print("FN: ", FN)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1: ", float(2*recall*precision)/float(recall + precision + 0.00000001))


In [17]:
class IntervalEvaluationSan(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.x_val, self.y_val = validation_data
        self.list_threshold = [0.0001, 0.001, 0.01, 0.1, 0.15, 0.2]

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=1)
            
            fineResDf = getSanFineInsRes(y_pred, val_ins_list)

            for threshold in self.list_threshold:
                print('THRESHOLD: ', threshold)
                docAnsDf = mergeDocumentResSan(fineResDf, val_id_df, threshold=threshold)
                sub = getSanSubmission(docAnsDf)
                getResult(gt_df, sub)

In [18]:
x1, x2, x3, y = preprocess_data(tran_ins_list, tokenizer_san, debug=True)

100%|██████████| 432850/432850 [24:21<00:00, 296.14it/s]


In [19]:
x_val1, x_val2, x_val3, y_val = preprocess_data(val_ins_list, tokenizer_san)

100%|██████████| 2755/2755 [00:09<00:00, 305.90it/s]


In [26]:
strategy = get_strategy()
# weight_path = '/content/drive/MyDrive/data/models/first-train-batch=128-lr=0.00005/weights-05.h5'
weight_path='/content/drive/MyDrive/data/models/san_1st/weights-18.h5'
with strategy.scope():
    sanModel = create_model_san(tokenizer_san, model_name)

    if os.path.isfile(weight_path):
        sanModel.predict([x1[0:1], x2[0:1], x3[0:1]])
        sanModel.load_weights(weight_path)

    opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
    lossSCE = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metricSCA = tf.keras.metrics.SparseCategoricalAccuracy()
    sanModel.compile(optimizer=opt, loss=lossSCE, metrics=[metricSCA], run_eagerly=False)

Running on TPU  ['10.3.233.218:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.3.233.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.3.233.218:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
INFO:absl:TPU has inputs with dynamic shapes:

In [27]:
save_locally = tf.train.CheckpointOptions(experimental_io_device='/job:localhost')

filepath="/content/drive/MyDrive/data/models/current/weights-{epoch:02d}.h5"
checkpoint_dir = os.path.dirname(filepath)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                verbose=1, 
                                                save_best_only=False, 
                                                save_weights_only=True,
                                                options=save_locally)
eval = IntervalEvaluationSan(validation_data=([x_val1, x_val2, x_val3], y_val))

In [ ]:
history = sanModel.fit(x=[x1, x2, x3],
                        y=y,
                        batch_size=128,
                        callbacks=[checkpoint, eval], 
                        epochs=100, 
                        validation_data=([x_val1, x_val2, x_val3], y_val), 
                        validation_batch_size=128)

Epoch 1/100


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]


   6/3382 [..............................] - ETA: 25:39 - loss: 1.3209 - sparse_categorical_accuracy: 0.6819WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0086s vs `on_train_batch_end` time: 10.0171s). Check your callbacks.


3382/3382 [==============================] - ETA: 0s - loss: 0.5439 - sparse_categorical_accuracy: 0.8071

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]


...

THRESHOLD:  0.0001
TP:  238
TN:  1220
FP:  1270
FN:  347
Recall:  0.4068376061421579
Precision:  0.1578249335823442
F1:  0.22742474491893788
THRESHOLD:  0.001
TP:  238
TN:  1220
FP:  1270
FN:  347
Recall:  0.4068376061421579
Precision:  0.1578249335823442
F1:  0.22742474491893788
THRESHOLD:  0.01
TP:  238
TN:  1220
FP:  1270
FN:  347
Recall:  0.4068376061421579
Precision:  0.1578249335823442
F1:  0.22742474491893788
THRESHOLD:  0.1
TP:  238
TN:  1220
FP:  1269
FN:  348
Recall:  0.40614334401682023
Precision:  0.15792966147449922
F1:  0.22742474491472844
THRESHOLD:  0.15
TP:  238
TN:  1221
FP:  1268
FN:  348
Recall:  0.40614334401682023
Precision:  0.15803452844752022
F1:  0.2275334565525228
THRESHOLD:  0.2
TP:  238
TN:  1222
FP:  1265
FN:  350
Recall:  0.4047619040735342
Precision:  0.1583499666278443
F1:  0.22764227216245256
Epoch 19/100
3382/3382 [==============================] - 1556s 460ms/step - loss: 0.3046 - sparse_categorical_accuracy: 0.8857 - val_loss: 8.3036 - val_sparse_ca

100%|██████████| 2755/2755 [00:14<00:00, 189.60it/s]


THRESHOLD:  0.0001
TP:  242
TN:  1222
FP:  1244
FN:  367
Recall:  0.39737274154782803
Precision:  0.16285329733320772
F1:  0.23102624863893922
THRESHOLD:  0.001
TP:  242
TN:  1222
FP:  1244
FN:  367
Recall:  0.39737274154782803
Precision:  0.16285329733320772
F1:  0.23102624863893922
THRESHOLD:  0.01
TP:  242
TN:  1222
FP:  1243
FN:  368
Recall:  0.396721310825047
Precision:  0.1629629628532236
F1:  0.23102624863494745
THRESHOLD:  0.1
TP:  242
TN:  1222
FP:  1243
FN:  368
Recall:  0.396721310825047
Precision:  0.1629629628532236
F1:  0.23102624863494745
THRESHOLD:  0.15
TP:  242
TN:  1222
FP:  1243
FN:  368
Recall:  0.396721310825047
Precision:  0.1629629628532236
F1:  0.23102624863494745
THRESHOLD:  0.2
TP:  242
TN:  1223
FP:  1242
FN:  368
Recall:  0.396721310825047
Precision:  0.1630727761704361
F1:  0.23113657635706333
Epoch 20/100
3382/3382 [==============================] - 1556s 460ms/step - loss: 0.2954 - sparse_categorical_accuracy: 0.8887 - val_loss: 8.1041 - val_sparse_categ

100%|██████████| 2755/2755 [00:15<00:00, 178.21it/s]


THRESHOLD:  0.0001
TP:  225
TN:  1269
FP:  1207
FN:  374
Recall:  0.3756260427785876
Precision:  0.15712290491821024
F1:  0.2215657267898145
THRESHOLD:  0.001
TP:  225
TN:  1269
FP:  1207
FN:  374
Recall:  0.3756260427785876
Precision:  0.15712290491821024
F1:  0.2215657267898145
THRESHOLD:  0.01
TP:  225
TN:  1269
FP:  1207
FN:  374
Recall:  0.3756260427785876
Precision:  0.15712290491821024
F1:  0.2215657267898145
THRESHOLD:  0.1
TP:  225
TN:  1270
FP:  1206
FN:  374
Recall:  0.3756260427785876
Precision:  0.1572327042926396
F1:  0.221674872468786
THRESHOLD:  0.15
TP:  225
TN:  1272
FP:  1202
FN:  376
Recall:  0.3743760393105224
Precision:  0.1576734406743704
F1:  0.22189348673488815
THRESHOLD:  0.2
TP:  225
TN:  1273
FP:  1200
FN:  377
Recall:  0.3737541522030662
Precision:  0.15789473673130192
F1:  0.2220029556446775
Epoch 21/100
 285/3382 [=>............................] - ETA: 23:40 - loss: 0.2883 - sparse_categorical_accuracy: 0.8889